# Extraindo informações sobre as turmas
Esse notebook tem como objetivo utilizar os datasets de matriculas e turmas para pegar gerar informações a respeito das turmas.

In [1]:
import pandas as pd
import numpy as np

In [2]:
matriculas = pd.read_csv("data/matricula-componente-20171.csv", sep=";")

In [3]:
# Pegando apenas as turmas de CeT
matriculas_cet = matriculas[matriculas.id_curso == 10320810]

In [4]:
matriculas_cet.descricao.unique()

array(['INDEFERIDO', 'APROVADO', 'APROVADO POR NOTA', 'CANCELADO',
       'REPROVADO', 'DESISTENCIA', 'TRANCADO',
       'REPROVADO POR MÉDIA E POR FALTAS', 'EXCLUIDA',
       'REPROVADO POR NOTA', 'EM ESPERA', 'REPROVADO POR FALTAS',
       'REPROVADO POR NOTA E FALTA'], dtype=object)

In [5]:
# Pegar apenas quem começou a pagar a materia

indesejada = ['DESISTENCIA', 'INDEFERIDO']

matriculas_cet = matriculas_cet[~matriculas_cet.descricao.isin(indesejada)]

In [6]:
# matriculas_cet[matriculas_cet.descricao == 'CANCELADO']

In [7]:
matriculas_cet.shape

(59610, 10)

In [8]:
matriculas_cet.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao'],
      dtype='object')

In [9]:
novo = matriculas_cet.drop_duplicates(subset=['id_turma', 'discente'])
novo = novo.dropna(subset=['media_final', 'numero_total_faltas'])

In [13]:
novo.shape
novo.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
455,57595855,0df99d841abe94466d686fe14e71ff34,10320810.0,1.0,8.7,f,0.0,8.7,0.0,APROVADO
1229,57598074,1e17512d2d5561ef70dc8c75443a18a6,10320810.0,3.0,8.9,f,0.0,8.2,10.0,APROVADO
1286,57599126,700f2a2dd2f5150c2638524ed5d22e1d,10320810.0,3.0,8.8,f,0.0,5.6,0.0,APROVADO POR NOTA
1289,57599126,4e763a0320aedffc0cb738a8b671e6ab,10320810.0,3.0,9.3,f,0.0,7.4,0.0,APROVADO
1292,57599126,5ff71bc1f0d4575cc05146aac69ad10c,10320810.0,2.0,0.0,f,0.0,0.0,0.0,CANCELADO


In [14]:
novo.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao'],
      dtype='object')

## Extraindo as informações estatísticas

In [15]:
# Calculando qnt_disc
qnt_disc = novo.groupby('id_turma', as_index=False)['media_final'].count() \
                    .rename(columns={'media_final': 'qnt_disc'})

# Calculando media_turma
media_turma = novo.groupby('id_turma', as_index=False)['media_final'].mean() \
                    .rename(columns={'media_final': 'media_turma'})

# Calculando media_faltas
media_faltas = novo.groupby('id_turma', as_index=False)['numero_total_faltas'].mean() \
                    .rename(columns={'numero_total_faltas': 'media_faltas'})

# Calculando qnt_reposicao
qnt_reposicao = novo[novo.reposicao == 't'].groupby('id_turma', as_index=False)['reposicao'].count() \
                    .rename(columns={'reposicao': 'qnt_reposicao'})

# Calculando qnt_aprovados
aprovados = ['APROVADO', 'APROVADO POR NOTA']
qnt_aprovados = novo[novo.descricao.isin(aprovados)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_aprovados'})

# Calculando qnt_reprovados
reprovado = ['REPROVADO', 'REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 
             'REPROVADO POR NOTA E FALTA', 'REPROVADO POR MÉDIA E POR FALTAS']
qnt_reprovados = novo[novo.descricao.isin(reprovado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_reprovados'})

# Calculando qnt_trancamento
trancado = ['CANCELADO', 'EXCLUIDA', 'TRANCADO']
qnt_trancamento = novo[novo.descricao.isin(trancado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_trancamento'})

In [16]:
# mesclando os datasets
turmas = pd.merge(left=qnt_disc, right=qnt_aprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_reprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_trancamento, how='outer')

In [17]:
turmas[turmas.qnt_disc > 100].head()

,id_turma,qnt_disc,qnt_aprovados,qnt_reprovados,qnt_trancamento
69,57595026,105,58.0,41.0,6.0
78,57595045,106,17.0,85.0,4.0
79,57595046,120,76.0,41.0,3.0
80,57595047,122,74.0,45.0,3.0
82,57595051,104,21.0,72.0,11.0


In [18]:
# Realizando o merge com as demais informações
turma_geral = pd.merge(left=turmas, right=media_turma, how='outer')
turma_geral = pd.merge(left=turma_geral, right=media_faltas, how='outer')
turma_geral = pd.merge(left=turma_geral, right=qnt_reposicao, how='outer')

# Preenchendo os valores vazios com 0
turma_geral = turma_geral.fillna(value=0)

In [19]:
turma_geral.to_csv('outputs/turmas_dados_2017_1.csv', sep=';', index=False)